## qa_ga_sim

This jn was written to present a Quality Assurement about the simulations, and comparing them to real objects.

All the parameters are listed in a JSON file in the same folder as this jupyter notebook.

Firstly, import packages and libraries.

In [ ]:
import numpy as np
from pathlib import Path
import healpy as hp
import json, os, sys, glob
from time import sleep
import tabulate
import astropy.coordinates as coord
import astropy.io.fits as fits
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy.io.fits import getdata

from qa_ga_sim.qa_ga_sim import (
    general_plots,
    plot_ftp,
    plots_ang_size,
    plot_err,
    plot_clusters_clean,
    plot_cmd_clean,
    plot_stellar_dens,
    export_results,
    read_iso
)

Below, load the JSON config file (change any parameter in case you want) and create folders for results.

See that there are two JSON files, one specifically for the qa_ga_sim and other from the code that builds the simulations.

In [ ]:
confg_qa = "qa_ga_sim.json"

with open(confg_qa) as fstream:
    param_qa = json.load(fstream)

globals().update(param_qa)

os.system('# cp ' + 'pars_simulations_file' + '.')

with open(pars_simulations_file) as fstream:
    param = json.load(fstream)

## Initial parameters

Now printing the parameters from simulations:

In [ ]:
for key, value in param.items():
    print('{:>30} | {:>10}'.format(key, value))

## Table of Simulated Objects

The cell below show the complete table of simulated clusters released in this simulation. An improved description of columns is provided:
<br>
<br>
0-HPX64: Ipix where the cluster is centered (Nested=True, Nside=64);
<br>
1-N: Star counts in cluster (before filtering stars from crowding);
<br>
2-MV: Absolute magnitude in V band (before filtering stars from crowding);
<br>
3-SNR: Poissonian Signal to Noise Ratio of the cluster. This is estimated by star counts within 2 arcmin over
<br>
root square of star counts within an annulus of rin=10 arcmin and rout = 25 arcmin, normalized
<br>
by area. This is calculated before filtering stars from crowding;
<br>
4-N_f: Star counts of filtering in stars by crowding;
<br>
5-MV_f: Absolute magnitude in V band after removing stars by crowding;
<br>
6-SNR_f: Signal to Noise Ratio calculated as described in (3) but after removing stars from crowding;
<br>
7-L: Galactic longitude (l), in degrees;
<br>
8-B: Galactic latitude (b), in degrees;
<br>
9-ra: Right Ascension (Equatorial coordinate), in degrees;
<br>
10-dec: Declination (Equatorial coordinate), in degrees;
<br>
11-r_exp: Exponential radius of cluster, in parsecs;
<br>
12-ell: Ellipticity (a - b) / a;
<br>
13-pa: Angular position (from North to East), in degrees;
<br>
14-mass: Visible mass of cluster (star accounted for mass are stars brighter than the limiting magnitude
<br>
of the simulation), in Solar masses;
<br>
15-dist: distance of the simulated cluster from Sun, in parsecs;

In [ ]:
with open(param['star_clusters_simulated']) as f:
    first_line = f.readline()

IPIX_with_clusters = np.loadtxt(param['star_clusters_simulated'], usecols=(0), dtype=int, unpack=True)
table = tabulate.tabulate(np.loadtxt(param['star_clusters_simulated'])[:][0:20],
                          tablefmt='html',
                          headers=(first_line[1:].split()))

print('Total of clusters simulated: {:d}\n'.format(len(IPIX_with_clusters)))

solid_angle = (np.sin(np.deg2rad(param['dec_max']))-np.sin(np.deg2rad(param['dec_min']))) * (np.deg2rad(param['ra_max']) -np.deg2rad(param['ra_min'])) * (180/np.pi) ** 2

print('Solid angle simulated: {:.2f} square degrees.\n'.format(solid_angle))

table

## Density of MW stars

In [ ]:
plot_stellar_dens(param)

## Isochrone

Now, reading information about the isochrone that was used to make simulated clusters.

In [ ]:
FeH_iso, logAge_iso, m_ini_iso, g_iso = read_iso(param['file_iso'])

print('[Fe/H]={:.2f}, Age={:.2f} Gyr'.format(FeH_iso, 10**(logAge_iso-9)))

## Global plots

### Absolute magnitude and size (half-light radius)

Plot features of simulated clusters and comparing them to a set of real objects.

Red circles (associated to 'Sim') are the features of objects taken into account initial set of stars.

Maroon circles are the cluster's features after removing stars from crowded fields (observational bias).

See that absolute magnitude increases (simulated objects are fainter) after filtering stars.

In [ ]:
general_plots(param['star_clusters_simulated'], param['output_plots'])

In [ ]:
plots_ang_size(param, FeH_iso)

In [ ]:
plot_ftp(param)

### Photometric errors (sample)

Plotting errors of stars in main magnitude band, comparing magnitudes from simulations and from catalog.


In [ ]:
plot_err(param['hpx_cats_clean_path'], 6)

### Individual clusters

Plot position of stars in clusters comparing filtered in and not filtered stars.

The region sampled is the center of the cluster where the crowding is more intense.

In [ ]:
if param['plot_individual']:
    ipix_cats = [(param['hpx_cats_clus_field'] + '/' + '{:d}'.format(i) + '.fits') for i in IPIX_with_clusters]
    ipix_clean_cats = [(param['hpx_cats_clean_path'] + '/' + '{:d}'.format(i) + '.fits') for i in IPIX_with_clusters]
                   
    plot_clusters_clean(param)

Figures below are 2D histograms of Color-Magnitude Diagrams (CMDs) plots comparing simulations.

On the left, all stars in a HealPixel (stars from simulated cluster and MW).
On the center, only stars from MW.
On the right, only stars from the simulated cluster.

In [ ]:
if param['plot_individual']:
    plot_cmd_clean(ipix_clean_cats[0:20], param['mmin'], param['mmax'], param['cmin'], param['cmax'],
                    'mag_g_with_err', 'mag_r_with_err', 'GC', param['output_plots'])